In [9]:
import os
import time
import json
import numpy as np
import pandas as pd
import wandb
import math
import random
from typing import Tuple, Sequence, Callable
from PIL import Image

import torch
import torch.optim as optim
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader
from torchinfo import summary
from torchvision import transforms
from torchvision.models import resnet50
import torch.nn.functional as F
from sklearn.preprocessing import LabelEncoder

from sole import MyDataset

In [2]:
#gpu 사용을 위해
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
transforms_train = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

transforms_test = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

trainset = MyDataset(dir='/home/compu/Documents/exports/sole_merge/train/',
                     image_ids='/home/compu/Documents/exports/newfile_sole_merge.json',
                     transforms=transforms_train)

testset = MyDataset(dir='/home/compu/Documents/exports/sole_merge/test/',
                    image_ids='/home/compu/Documents/exports/newfile_sole_merge.json',
                    transforms=transforms_test)

train_loader = DataLoader(trainset, batch_size=64, num_workers=8)
test_loader = DataLoader(testset, batch_size=32, num_workers=4)

print('학습 데이터셋 크기:', len(trainset))
print('테스트 데이터셋 크기:', len(testset))


학습 데이터셋 크기: 15513
테스트 데이터셋 크기: 4192


In [4]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [7]:
class MyModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.resnet = resnet50(pretrained=True)
        self.classifier = nn.Linear(1000, 4)

    def forward(self, x):
        x = self.resnet(x)
        x = self.classifier(x)

        return x

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MyModel().to(device)
print(summary(model, input_size=(1, 3, 128, 128), verbose=0))

Layer (type:depth-idx)                        Output Shape              Param #
MyModel                                       [1, 4]                    --
├─ResNet: 1-1                                 [1, 1000]                 --
│    └─Conv2d: 2-1                            [1, 64, 64, 64]           9,408
│    └─BatchNorm2d: 2-2                       [1, 64, 64, 64]           128
│    └─ReLU: 2-3                              [1, 64, 64, 64]           --
│    └─MaxPool2d: 2-4                         [1, 64, 32, 32]           --
│    └─Sequential: 2-5                        [1, 256, 32, 32]          --
│    │    └─Bottleneck: 3-1                   [1, 256, 32, 32]          75,008
│    │    └─Bottleneck: 3-2                   [1, 256, 32, 32]          70,400
│    │    └─Bottleneck: 3-3                   [1, 256, 32, 32]          70,400
│    └─Sequential: 2-6                        [1, 512, 16, 16]          --
│    │    └─Bottleneck: 3-4                   [1, 512, 16, 16]          379,392

In [10]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MultiLabelSoftMarginLoss()
wandb.init(project="kristin")
wandb.watch(model, criterion, log='all', log_freq=10)

num_epochs = 100
model.train()
start_time = time.time()

# early_stopping 객체 선언(5번의 epoch 연속으로 loss 미개선 시에 조기 종료 예시)
early_stopping = EarlyStopping(patience = 10, verbose = True, path = "material1.pt")

for epoch in range(num_epochs):
    
    for i, (images, targets) in enumerate(train_loader):
        optimizer.zero_grad()

        images = images.to(device)
        targets = targets.to(device)

        outputs = model(images)
        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()

        if (i+1) % 64 == 0:
            outputs = (outputs > 0.5).float()
            acc = (outputs == targets).float().mean()
            metrics = {"train_loss": loss.item(), 
                       "train_accuracy": acc.item()}
            wandb.log(metrics)

    early_stopping(loss.item(), model) # 현재 과적합 상황 추적
    
    if early_stopping.early_stop: # 조건 만족 시 조기 종료
        break

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/home/compu/.local/lib/python3.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
wandb: Currently logged in as: kew118. Use `wandb login --relogin` to force relogin


Validation loss decreased (inf --> 0.356663).  Saving model ...
Validation loss decreased (0.356663 --> 0.312042).  Saving model ...
Validation loss decreased (0.312042 --> 0.232291).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
EarlyStopping counter: 4 out of 10
Validation loss decreased (0.232291 --> 0.223788).  Saving model ...
EarlyStopping counter: 1 out of 10
Validation loss decreased (0.223788 --> 0.221102).  Saving model ...
EarlyStopping counter: 1 out of 10
Validation loss decreased (0.221102 --> 0.210657).  Saving model ...
EarlyStopping counter: 1 out of 10
Validation loss decreased (0.210657 --> 0.194601).  Saving model ...
EarlyStopping counter: 1 out of 10
Validation loss decreased (0.194601 --> 0.177527).  Saving model ...
EarlyStopping counter: 1 out of 10
Validation loss decreased (0.177527 --> 0.162767).  Saving model ...
EarlyStopping counter: 1 out of 10
Validation loss decreased (0.16276

In [11]:
model.eval()
start_time = time.time()

with torch.no_grad():
    preds_list = []
    y_list = []
    
    for i, (images, targets) in enumerate(test_loader):
        images = images.to(device)
        targets = targets.to(device)
        outputs = model(images)
        loss = criterion(outputs, targets)

        outputs = (outputs > 0.5).float()
        acc = (outputs == targets).float().mean()
        
        preds_list.append(outputs.float().cpu())
        y_list.append(targets.cpu())
        print(f'{i+1}: {loss.item():.5f}, {acc.item():.5f}, {time.time() - start_time}')
        val_metrics = {"test_loss": loss.item(), 
                       "test_accuracy": acc.item()}
        wandb.log({**metrics, **val_metrics})
    
    preds_list = np.concatenate(preds_list)
    y_list = np.concatenate(y_list)
    

wandb.finish()

1: 0.23652, 0.92969, 0.28695225715637207
2: 0.51742, 0.87500, 0.3134310245513916
3: 0.48398, 0.86719, 0.694744348526001
4: 0.82765, 0.86719, 0.7150051593780518
5: 0.52219, 0.92188, 0.736314058303833
6: 0.50478, 0.90625, 0.7595958709716797
7: 0.41541, 0.91406, 0.7814106941223145
8: 0.39483, 0.89062, 0.8023965358734131
9: 0.36968, 0.91406, 0.8242251873016357
10: 0.42715, 0.90625, 0.846113920211792
11: 0.31251, 0.91406, 0.8651473522186279
12: 0.58305, 0.88281, 0.8970885276794434
13: 0.46422, 0.88281, 1.2192354202270508
14: 0.55252, 0.87500, 1.2377114295959473
15: 0.52161, 0.88281, 1.2554545402526855
16: 0.76843, 0.82812, 1.2736713886260986
17: 1.12615, 0.85938, 1.2927753925323486
18: 0.52563, 0.89062, 1.3106091022491455
19: 0.41853, 0.90625, 1.3298544883728027
20: 0.40442, 0.92188, 1.3491060733795166
21: 0.24611, 0.92969, 1.3684618473052979
22: 0.62828, 0.87500, 1.387753963470459
23: 0.41960, 0.91406, 1.8078458309173584
24: 0.39642, 0.90625, 1.827584981918335
25: 0.56296, 0.89062, 1.84616

test_accuracy,▂▂▃▄▂▃▅▄▅▅▂▃▅▁▇▄▁▆▄▃█▇▃▂▄▆▄▄▅▂▆▄▄▆▃▂▂▆▆▄
test_loss,▅▇▃▄▅▅▄▃▃▃▅▆▁█▂▃▆▃▅▄▁▂▄▅▃▃▄▄▄▆▃▄▄▂▅▆▆▂▃▄
train_accuracy,▁▄▄▃▅▅▃▅▅▆▆▇▇▆█▇▇▇▇█▇██▇████████████████
train_loss,█▅▅▆▄▄▅▄▃▃▄▂▂▃▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,0.92188
test_loss,0.19394
train_accuracy,1.0
train_loss,0.00783


In [14]:
from sklearn.metrics import precision_score, recall_score, f1_score

# 정확도(accuracy) 계산
accuracy = np.mean(y_list == preds_list)
print(f'Accuracy: {accuracy:.3f}')

# 정밀도(precision) 계산
precision = precision_score(y_list, preds_list, average='micro')
print(f'Precision: {precision:.3f}')

# 재현율(recall) 계산
recall = recall_score(y_list, preds_list, average='micro')
print(f'Recall: {recall:.3f}')

# F1 점수(F1 score) 계산
f1 = f1_score(y_list, preds_list, average='micro')
print(f'F1: {f1:.3f}')

Accuracy: 0.901
Precision: 0.848
Recall: 0.831
F1: 0.839


In [15]:
import sklearn.metrics as skm

print(skm.classification_report(y_list, preds_list))

              precision    recall  f1-score   support

           0       0.82      0.94      0.87      2107
           1       0.81      0.60      0.69      1250
           2       0.75      0.58      0.65       266
           3       0.93      0.91      0.92      1599

   micro avg       0.85      0.83      0.84      5222
   macro avg       0.83      0.76      0.78      5222
weighted avg       0.85      0.83      0.83      5222
 samples avg       0.84      0.84      0.83      5222



/home/compu/anaconda3/envs/kristin/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
# 모델 저장
torch.save(model.state_dict(), 'sole_end.pt')

아래 코드부터는 모델 불러오기

In [ ]:
# 모델 불러오기 전 정의
class MyModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.resnet = resnet50(pretrained=True)
        self.classifier = nn.Linear(1000, 7)

    def forward(self, x):
        x = self.resnet(x)
        x = self.classifier(x)

        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MyModel().to(device)
print(summary(model, input_size=(1, 3, 128, 128), verbose=0))

C:\Users\Admin\anaconda3\envs\seongin\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Admin\anaconda3\envs\seongin\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Layer (type:depth-idx)                        Output Shape              Param #
MyModel                                       [1, 7]                    --
├─ResNet: 1-1                                 [1, 1000]                 --
│    └─Conv2d: 2-1                            [1, 64, 64, 64]           9,408
│    └─BatchNorm2d: 2-2                       [1, 64, 64, 64]           128
│    └─ReLU: 2-3                              [1, 64, 64, 64]           --
│    └─MaxPool2d: 2-4                         [1, 64, 32, 32]           --
│    └─Sequential: 2-5                        [1, 256, 32, 32]          --
│    │    └─Bottleneck: 3-1                   [1, 256, 32, 32]          75,008
│    │    └─Bottleneck: 3-2                   [1, 256, 32, 32]          70,400
│    │    └─Bottleneck: 3-3                   [1, 256, 32, 32]          70,400
│    └─Sequential: 2-6                        [1, 512, 16, 16]          --
│    │    └─Bottleneck: 3-4                   [1, 512, 16, 16]          379,392

In [ ]:
# 모델 불러오기
model = MyModel()
model.load_state_dict(torch.load('sole1.pt'))

<All keys matched successfully>

In [ ]:
model.eval()
start_time = time.time()

with torch.no_grad():
    preds_list = []
    y_list = []
    
    for i, (images, targets) in enumerate(test_loader):
        images = images.to(device)
        targets = targets.to(device)
        outputs = model(images)
        loss = criterion(outputs, targets)
        
        print(f'[예측 결과: {outputs[0]}] (실제 정답: {targets.data[0]})')
        
        if (i+1) % 10 == 0:
            outputs = outputs > 0.5
            acc = (outputs == targets).float().mean()
            preds_list.append(outputs.float())
            y_list.append(targets)
            print(f'{i+1}: {loss.item():.5f}, {acc.item():.5f}, {time.time() - start_time}')
    
    preds_list = np.concatenate(preds_list)
    y_list = np.concatenate(y_list)

[예측 결과: tensor([ -7.1254,  16.5854,  -7.2982, -23.1468, -12.2286, -14.2949, -14.1143])] (실제 정답: tensor([0., 1., 0., 0., 0., 0., 0.]))
[예측 결과: tensor([-11.0022, -11.3789,  -9.2108, -12.9680, -11.5576, -13.8835,  11.9744])] (실제 정답: tensor([0., 0., 0., 0., 0., 0., 1.]))
[예측 결과: tensor([ -9.6251,   4.8976,   5.9598, -12.9161, -12.8990,  -7.4001, -11.2430])] (실제 정답: tensor([0., 1., 1., 0., 0., 0., 0.]))
[예측 결과: tensor([  0.6248,  13.1945, -14.9475, -22.4007,  -7.8260, -15.3666, -11.5244])] (실제 정답: tensor([1., 1., 0., 0., 0., 0., 0.]))
[예측 결과: tensor([ -6.2745,   8.3974,   1.7113, -12.7735, -11.2447,  -6.9007,  -9.1765])] (실제 정답: tensor([0., 1., 1., 0., 0., 0., 0.]))
[예측 결과: tensor([ -4.8020,   7.3870,  -4.6649, -14.1674,  -8.5417,  -8.9812,  -6.0899])] (실제 정답: tensor([0., 1., 0., 0., 0., 0., 0.]))
[예측 결과: tensor([-11.3798,   6.7946,  -7.0100, -18.8097, -10.0640, -13.3474,  -4.9412])] (실제 정답: tensor([0., 1., 0., 0., 0., 0., 0.]))
[예측 결과: tensor([ -5.8625,   4.5242,   2.3362, -10.4199, -10.43

ValueError: need at least one array to concatenate

In [ ]:
outputs

tensor([[-13.5821, -11.6343, -12.4343,  ..., -11.5689, -16.1081,  12.5219],
        [ -5.8308,   3.1381,  -4.6446,  ...,  -8.7523,  -9.0775,  -1.6631],
        [ -4.2882,   2.3651,  -1.6322,  ...,  -7.1041,  -4.4521,  -4.2419],
        ...,
        [ -8.4108,   5.9161,  -4.1284,  ...,  -8.7688,  -8.4205,  -4.5192],
        [ -4.9900,   7.4485,   2.8699,  ..., -13.4122,  -5.7267, -12.2228],
        [ -9.3158,   6.5222,   0.8218,  ..., -12.2816,  -9.3427,  -7.3485]])

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# 정확도(accuracy) 계산
accuracy = np.mean(y_list == preds_list)
print(f'Accuracy: {accuracy:.3f}')

# 정밀도(precision) 계산
precision = precision_score(y_list, preds_list, average='micro')
print(f'Precision: {precision:.3f}')

# 재현율(recall) 계산
recall = recall_score(y_list, preds_list, average='micro')
print(f'Recall: {recall:.3f}')

# F1 점수(F1 score) 계산
f1 = f1_score(y_list, preds_list, average='micro')
print(f'F1: {f1:.3f}')